In [11]:
%pip install scikit-learn

import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [12]:
def detect_coins(image):
    gray = cv2.GaussianBlur(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (9, 9), 2)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100,
                               param1=100, param2=70, minRadius=40, maxRadius=120)

    annotated, total, counts = image.copy(), 0, {1: 0, 5: 0, 10: 0, 20: 0}
    if circles is None: return annotated, counts, total

    for x, y, r in np.uint16(np.around(circles[0])):
        roi = image[y-r:y+r, x-r:x+r]
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        gold = cv2.inRange(hsv, (10, 40, 50), (40, 255, 255))
        gold_ratio = np.mean(gold > 0)

        if gold_ratio > 0.02: val = 20
        elif r < 55: val = 1
        elif r < 70: val = 5
        else: val = 10

        counts[val] += 1
        total += val

        cv2.circle(annotated, (x, y), r, (0, 255, 0), 2)
        cv2.putText(annotated, f" PHP {val}", (x-25, y), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (0,0,255), 2)

    return annotated, counts, total


In [13]:
def detect_bills(image):
    hsv, annotated = cv2.cvtColor(image, cv2.COLOR_BGR2HSV), image.copy()
    total, counts = 0, {20:0, 50:0, 100:0, 200:0, 500:0, 1000:0}

    bill_colors = {
        20: ([5, 100, 100],   [20, 255, 255]),   # orange
        50: ([160, 100, 100], [179, 255, 255]),  # red
        100: ([125, 80, 80],  [150, 255, 255]),  # violet
        200: ([40, 80, 80],   [80, 255, 255]),   # green
        500: ([20, 80, 80],   [35, 255, 255]),   # yellow
        1000:([90, 80, 80],   [120, 255, 255])   # blue
    }

    for val, (low, high) in bill_colors.items():
        mask = cv2.inRange(hsv, np.array(low), np.array(high))
        if cv2.countNonZero(mask) > 5000:   # basic threshold
            counts[val] += 1
            total += val
            cv2.putText(annotated, f"PHP {val}", (50, 50+30*list(bill_colors).index(val)),
                        cv2.FONT_HERSHEY_TRIPLEX, 1, (0,0,255), 2)

    return annotated, counts, total


In [14]:
def detect_money(image):
    coins_img, coin_counts, coin_total = detect_coins(image)
    bills_img, bill_counts, bill_total = detect_bills(coins_img)

    total = coin_total + bill_total
    counts = {**coin_counts, **bill_counts}

    cv2.putText(bills_img, f"TOTAL: {total}", (30, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3)

    return bills_img, counts, total



In [15]:
class Activity4:
    def __init__(self, root, cam_index=0):
        self.root, self.cam_index = root, cam_index
        self.cap, self.frame, self.running = None, None, False

        self.video_label = tk.Label(root); self.video_label.pack()
        tk.Button(root, text="Start", command=self.start).pack(side="left", padx=5)
        tk.Button(root, text="Stop", command=self.stop).pack(side="left", padx=5)
        tk.Button(root, text="Snapshot", command=self.snapshot).pack(side="left", padx=5)

    def start(self):
        if self.running: return
        self.cap = cv2.VideoCapture(self.cam_index)
        if not self.cap.isOpened(): print("No camera"); return
        self.running = True; self.update()

    def stop(self):
        if self.cap: self.cap.release()
        self.running = False

    def snapshot(self):
        if self.frame is not None:
            cv2.imwrite("snapshot.png", cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB))
            print("Saved snapshot")

    def update(self):
        if self.running and self.cap:
            ret, frame = self.cap.read()
            if ret:
                img, _, _ = detect_money(frame)
                self.frame = img
                imgtk = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
                self.video_label.config(image=imgtk); self.video_label.imgtk = imgtk
        self.root.after(30, self.update)


In [17]:
root = tk.Tk()
app = Activity4(root)
root.mainloop()